In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import jax_dataclasses as jdc
from functools import partial

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *
from sdf_world.sparse_ipopt import *

from flax import linen as nn
from flax.training import orbax_utils
import orbax
import pickle
import time

### Load Models

In [2]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
restored_grasp = orbax_checkpointer.restore("model/grasp_net_prob_dist")
restored_manip = orbax_checkpointer.restore("model/manip_net_posevec")

#grasp net
grasp_net = GraspNet(32)
grasp_fn = lambda x: grasp_net.apply(restored_grasp["params"], x)

grasp_logit_fn = lambda g: grasp_fn(g)[0]
grasp_dist_fn = lambda g: grasp_fn(g)[1]
#manip net
manip_net = ManipNet(64)
manip_fn = lambda x: manip_net.apply(restored_manip["params"], x)[0]

### Load world

In [3]:
world = SDFWorld()
world.show_in_jupyter()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
# robot, hand
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

hand_model = RobotModel(HAND_URDF, PANDA_PACKAGE, True)
for link_name, link in hand_model.links.items():
    link.set_surface_points(10)
hand = Robot(world.vis, "hand1", hand_model, color="yellow", alpha=0.5)

In [5]:
#load sdf meshes
table_lengths = [0.4, 0.4, 0.2]
table_start = Box(world.vis, "table_start", table_lengths, 'white', 0.5)
table_goal = Box(world.vis, "table_goal", table_lengths, 'white', 0.5)
obj_start = Mesh(world.vis, "obj_start", 
                 "./sdf_world/assets/object/mesh.obj",
                 color="blue", alpha=0.5)
obj_goal = Mesh(world.vis, "obj_goal", 
                "./sdf_world/assets/object/mesh.obj",
                color="green", alpha=0.5)

In [6]:
table_start.set_translate([0.5, -0.3, 0.2/2])
table_goal.set_translate([0.5, 0.3, 0.2/2])
obj_lengths = obj_start.mesh.bounding_box.primitive.extents
obj_start.set_translate([0.5, -0.3, obj_lengths[-1]/2+table_lengths[-1]])
trans_goal = jnp.array([0.5, 0.3, obj_lengths[-2]/2+table_lengths[-1]])
obj_goal_pose = SE3.from_rotation_and_translation(
    SO3.from_rpy_radians(jnp.pi/2, 0,0), trans_goal)
obj_goal.set_pose(obj_goal_pose)

In [352]:
to_posevec = lambda x: jnp.hstack([x[4:], SO3(x[:4]).log()])
to_wxyzxyz = lambda x: jnp.hstack([SO3.exp(x[3:]).parameters(), x[:3]])
env = SDFContainer([table_start, table_goal], 0.05)
def grasp_reconst(grasp:Array):
    rot = SO3(grasp_fn(grasp)[2:]).normalize()
    trans = grasp/restored_grasp["scale_to_norm"]
    return SE3.from_rotation_and_translation(rot, trans)

#visualization
hand_pc = hand.get_surface_points_fn(jnp.array([0.04, 0.04]))
hand_pose_wrt_ee = SE3.from_translation(jnp.array([0,0,-0.105]))
@jax.jit
def get_hand_pc(grasp, posevec):
    grasp_pose = grasp_reconst(grasp)
    hand_base_pose_wrt_world = SE3(to_wxyzxyz(posevec)) @ grasp_pose @ hand_pose_wrt_ee
    assigned_hand_pc = jax.vmap(hand_base_pose_wrt_world.apply)(hand_pc)
    return assigned_hand_pc

pc_hand = PointCloud(world.vis, "hand_pc", np.zeros((100,3)), color="red", size=0.01)

In [353]:
# Kinematics
def get_rotvec_angvel_map(v):
    def skew(v):
        v1, v2, v3 = v
        return jnp.array([[0, -v3, v2],
                        [v3, 0., -v1],
                        [-v2, v1, 0.]])
    def identity(v, vmag):
        return np.eye(3)
    def angvel_to_ecvel(v, vmag):
        vskew = skew(v)
        term3 = vskew@vskew * 1/vmag**2 * (1-vmag/2 * jnp.sin(vmag)/(1-jnp.cos(vmag)))
        return jnp.eye(3) - 1/2*skew(v) + term3
    vmag = jnp.linalg.norm(v)
    return jax.lax.cond(vmag < 1e-3, identity, angvel_to_ecvel, v, vmag)
    
    

@jax.jit
def get_ee_fk_jac(q):
    # outputs ee_posevec and analytical jacobian
    fks = panda_model.fk_fn(q)
    ee = SE3(fks[-1])
    p_ee = fks[-1][-3:]
    #rotvec_ee = SO3(fks[-1][:4]).log()
    #E = get_rotvec_angvel_map(rotvec_ee)
    jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        jac.append(jnp.hstack([lin_vel, rot_axis]))
    jac = jnp.array(jac).T #geometric jacobian
    #jac = jac.at[3:, :].set(E @ jac[3:, :])
    return ee, jac

### Prepare functions

In [354]:
#constr fns
def grasp_constr_fn(grasp):
    return grasp_logit_fn(grasp)

def manip_constr_fn(grasp, posevec):
    obj_pose = SE3(to_wxyzxyz(posevec))
    grasp_pose = obj_pose @ grasp_reconst(grasp)
    zflip = SE3.from_rotation(SO3.from_z_radians(jnp.pi))
    grasp_pose_flip = grasp_pose @ zflip
    posevecs = [to_posevec(pose.parameters()) for pose in [grasp_pose, grasp_pose_flip]]
    return jax.vmap(manip_fn)(jnp.vstack(posevecs)).max()

def dist_constr_fn(g1, posevec_st, posevec_ed):
    grasps = jnp.vstack([g1, g1])
    obj_poses = jnp.vstack([posevec_st, posevec_ed])
    pcs = jax.vmap(get_hand_pc, (0,0))(grasps, obj_poses)
    distances = env.distances(jnp.vstack(pcs))
    top4_indices = jnp.argpartition(distances, 4)[:4]
    return distances[top4_indices]

grasp_constr_jac_fns = [jax.jacrev(grasp_constr_fn)]
manip_constr_jac_fns = [jax.jacrev(manip_constr_fn, argnums=argnums) for argnums in [0,1]]
dist_constr_jac_fns = [jax.jacrev(dist_constr_fn)] + [None]*2

ws_lb = np.array([-1,-1,-0.5, -np.pi, -np.pi, -np.pi])
ws_ub = np.array([1,1,1.5, np.pi, np.pi, np.pi])

In [355]:
import time
builder = SparseIPOPTBuilder()

builder.add_variable("g_pick", 3, -1., 1.)
builder.add_parameter("p_start", to_posevec(obj_start.pose.parameters()))
builder.add_parameter("p_goal", to_posevec(obj_goal.pose.parameters()))

builder.register_fn("grasp_constr", [3], 1,
                          grasp_constr_fn, grasp_constr_jac_fns)
builder.register_fn("manip_constr", [3, 6], 1,
                          manip_constr_fn, manip_constr_jac_fns)
builder.register_fn("dist_constr", [3, 6, 6], 4,
                          dist_constr_fn, dist_constr_jac_fns)

def debug_obj(g_pick, posevec_st, posevec_ed):
    grasps = jnp.vstack([g_pick, g_pick])
    obj_poses = jnp.vstack([posevec_st, posevec_ed])
    pcs = jax.vmap(get_hand_pc, (0,0))(grasps, obj_poses)
    pc_hand.reload(points=np.vstack(pcs), color="red")
    time.sleep(0.1)
    return 0.
builder.register_fn("debug_obj", [3, 6, 6], 1,
                    debug_obj, [None, None, None])

builder.add_objective(["g_pick", "p_start", "p_goal"], "debug_obj")

builder.add_constr("grasp_prob", ["g_pick"], "grasp_constr",
                   1., np.inf)
builder.add_constr("manip_pick", ["g_pick", "p_start"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("manip_place", ["g_pick", "p_goal"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("dist", ["g_pick", "p_start", "p_goal"], "dist_constr",
                   0.05, np.inf)

builder.freeze()

In [446]:
debug_obj(xsol, builder.param_dict["p_start"], builder.param_dict["p_goal"])

0.0

In [356]:
ipopt = builder.build(compile_obj=False)


Sparsity pattern:
00 01 02 
03 04 05 
06 07 08 
09 10 11 
12 13 14 
15 16 17 
18 19 20 


In [389]:
xlist = []

In [394]:
xlist.append(xinit)

In [404]:
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("acceptable_tol", 1.) #release
ipopt.add_option("acceptable_obj_change_tol", 0.01)
ipopt.add_option("acceptable_dual_inf_tol", 1.) 
ipopt.add_option('mu_strategy', 'adaptive')

xinit = np.random.uniform(-1, 1, size=3)
xsol, info = ipopt.solve(xlist[1])

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       21
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        7
        inequality constraints with only lower bounds:        7
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 4.99e+02 1.01e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [359]:
grasp = xsol

In [155]:
frame = Frame(world.vis, "frame")
frame2 = Frame(world.vis, "frame2")

In [14]:
q = panda.neutral
panda.set_joint_angles(q)
target_pose = (obj_start.pose @ grasp_reconst(grasp))

In [15]:
def kin_error_fn(q, grasp, p_obj):
    obj_pose = SE3(to_wxyzxyz(p_obj))
    target_pose = obj_pose @ grasp_reconst(grasp)
    ee_pose, _ = get_ee_fk_jac(q)
    # represented in body
    R_ee = ee_pose.rotation().as_matrix()
    err_pos = R_ee.T@(target_pose.translation() - ee_pose.translation())
    err_rot = ee_pose.rotation().inverse() @ target_pose.rotation()
    err = jnp.hstack([err_pos, err_rot.log()])
    return err

def kin_error_jac_fn(q, grasp, p_obj):
    ee_pose, geom_jac = get_ee_fk_jac(q)
    obj_pose = SE3(to_wxyzxyz(p_obj))
    target_pose = obj_pose @ grasp_reconst(grasp)
    err_rot = ee_pose.rotation().inverse() @ target_pose.rotation()
    R_ee = ee_pose.rotation().as_matrix()
    B = get_rotvec_angvel_map(err_rot.log())
    jac_pos = - R_ee.T @ geom_jac[:3]
    jac_rot = - B @ R_ee.T @ geom_jac[3:]
    jac = jnp.vstack([jac_pos, jac_rot])
    return jac


num_q_mid = 1
num_traj = num_q_mid*3+2
idx_pick = num_q_mid
idx_place = idx_pick + num_q_mid + 1
q0 = panda.neutral
def min_dist(*qs):
    qs = jnp.vstack([q0, *qs, q0])
    qdiff = qs[1:] - qs[:-1]
    return 0.5*jnp.sum(qdiff.flatten()**2)
grads_min_dist = [jax.grad(min_dist, argnums=i) for i in range(num_traj)]

def debug_obj2(*qs):
    panda.set_joint_angles(qs[0])
    # posevec_ee, jac = get_ee_fk_jac(q)
    # obj_pose = SE3(to_wxyzxyz(p_obj))
    # frame2.set_pose(SE3(to_wxyzxyz(posevec_ee)))
    # grasp_pose = obj_pose @ grasp_reconst(grasp)
    # frame.set_pose(grasp_pose)
    # pc.reload(points=np.vstack(pcs))
    time.sleep(0.1)
    return min_dist(*qs)


builder2 = SparseIPOPTBuilder()

for i in range(num_traj):
    idx = i+1
    builder2.add_variable(f"q{idx}", 7, panda.lb, panda.ub)
builder2.add_parameter("p_start", to_posevec(obj_start.pose.parameters()))
builder2.add_parameter("p_goal", to_posevec(obj_goal.pose.parameters()))
builder2.add_parameter("grasp", xsol)

In [463]:
builder2.register_fn("kin_constr", [7, 3, 6], 6,
                          kin_error_fn, [kin_error_jac_fn, None, None])
builder2.register_fn("obj", [7]*num_traj, 1,
                          debug_obj2, grads_min_dist)

In [464]:
builder2.add_objective(list(builder2.x_info.keys()), "obj")
builder2.add_constr("kin_pick", [f"q{idx_pick+1}", "grasp", "p_start"], "kin_constr",
                   0., 0.)
builder2.add_constr("kin_place", [f"q{idx_place+1}", "grasp", "p_goal"], "kin_constr",
                    0., 0.)

In [465]:
builder2.freeze()
ipopt2 = builder2.build(compile_obj=False)


Sparsity pattern:
-- -- -- -- -- -- -- 00 01 02 03 04 05 06 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- 07 08 09 10 11 12 13 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- 14 15 16 17 18 19 20 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- 21 22 23 24 25 26 27 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- 28 29 30 31 32 33 34 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- 35 36 37 38 39 40 41 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 42 43 44 45 46 47 48 -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 49 50 51 52 53 54 55 -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 56 57 58 59 60 61 62 -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- -- 

In [475]:
ipopt2.add_option("acceptable_iter", 2)
ipopt2.add_option("acceptable_tol", 0.1) #release
ipopt2.add_option("acceptable_obj_change_tol", 0.0001)
ipopt2.add_option("acceptable_dual_inf_tol", 1.) 
ipopt2.add_option('mu_strategy', 'adaptive')

qs_init = jnp.tile(panda.neutral, 5)
qsol, info = ipopt2.solve(qs_init)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       84
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       35
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       35
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.56e+00 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [469]:
qs = qsol.reshape(-1, 7)

In [481]:
panda.set_joint_angles(qs[0])

In [544]:
qs = qsol.reshape(-1, 7)

In [ ]:
def distance_fn(*qs):
    body_points = jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
    
    return err

def kin_error_jac_fn(q, grasp, p_obj):
    ee_pose, geom_jac = get_ee_fk_jac(q)
    obj_pose = SE3(to_wxyzxyz(p_obj))
    target_pose = obj_pose @ grasp_reconst(grasp)
    err_rot = ee_pose.rotation().inverse() @ target_pose.rotation()
    R_ee = ee_pose.rotation().as_matrix()
    B = get_rotvec_angvel_map(err_rot.log())
    jac_pos = - R_ee.T @ geom_jac[:3]
    jac_rot = - B @ R_ee.T @ geom_jac[3:]
    jac = jnp.vstack([jac_pos, jac_rot])
    return jac

In [ ]:
def dist_constr_fn(g1, posevec_st, posevec_ed):
    grasps = jnp.vstack([g1, g1])
    obj_poses = jnp.vstack([posevec_st, posevec_ed])
    pcs = jax.vmap(get_hand_pc, (0,0))(grasps, obj_poses)
    distances = env.distances(jnp.vstack(pcs))
    top4_indices = jnp.argpartition(distances, 4)[:4]
    return distances[top4_indices]

In [600]:
def exact_dist_constr_fn(*qs):
    qs = jnp.vstack(qs)
    hold_indices = np.array([2])
    body_points = jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
    _obj_points = jnp.vstack(jax.vmap(get_object_points)(qs[hold_indices]))
    points = jnp.vstack([body_points, _obj_points])
    distances = env.distances(points)
    top4_indices = jnp.argpartition(distances, 4)[:4]
    #visualize
    body_pc.reload(points=body_points, size=0.01)
    obj_pc.reload(points=_obj_points)
    return distances[top4_indices]

In [17]:
body_pc = PointCloud(world.vis, "body_pc", body_points, 0.01, "blue")

In [18]:
hold_indices = np.array([2])
qs = np.random.uniform(panda.lb, panda.ub, size=(10, 7))
body_points = jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
distances = env.distances(body_points)
top4_indices = jnp.argpartition(distances, 4)[:4]
body_pc.reload(points=body_points)

In [19]:
num_robot_points = 200
num_link_points = 20

q_idx = 0
top4_indices
q_indices = top4_indices // (num_robot_points)
link_indices = (top4_indices % (num_robot_points)) // (num_link_points)

Array([5, 2, 4, 5], dtype=int32)

In [28]:
idx = 0
idx = top4_indices[0]

array([[-2.70112616, -1.62282564,  1.93425929, -0.29548673,  1.50865186,
         2.92775226, -0.0541662 ],
       [-2.70112616, -1.62282564,  1.93425929, -0.29548673,  1.50865186,
         2.92775226, -0.0541662 ],
       [-2.70112616, -1.62282564,  1.93425929, -0.29548673,  1.50865186,
         2.92775226, -0.0541662 ],
       [-2.70112616, -1.62282564,  1.93425929, -0.29548673,  1.50865186,
         2.92775226, -0.0541662 ]])

In [34]:
q = qs[q_indices[idx]]
link_idx = link_indices[idx]
point = body_points[top4_indices[idx]]

In [46]:
def grad(q, point, link_idx):
    point_jac = point_jacobian(q, point, link_idx)
    repulsive_grad = jax.grad(env.distance)(point)
    return repulsive_grad @ point_jac
jax.vmap(grad)(qs[q_indices], body_points[top4_indices], link_indices)

Array([[ 0.        ,  0.6725969 , -0.11665344,  0.12553938, -0.01713071,
        -0.02625567,  0.        ],
       [-0.33804867, -0.06244549, -0.11472226, -0.03939329, -0.11984427,
        -0.09958047,  0.        ],
       [ 0.        ,  0.71785617, -0.08789232,  0.14302959,  0.02254761,
         0.00635397,  0.        ],
       [-0.00839625,  0.7350494 , -0.09208698,  0.14283697,  0.02348387,
         0.02148601,  0.        ]], dtype=float32)

In [24]:
point = body_points[top4_indices[3]]
jac_point = point_jacobian()
jax.grad(env.distance)()

Array([0.03654607, 0.        , 0.99933195], dtype=float32)

In [26]:
idx = top4_indices[0]
point = body_points[idx]
num_robot_points = 200
num_link_points = 20
#q_idx = idx // (num_robot_points)
# link_idx = (idx % (num_robot_points)) // (num_link_points)
# point_idx = ((idx % (200)) % (20))

def point_jacobian(q, point, link_idx):
    q_idx = idx // (num_robot_points)
    link_idx = (idx % (num_robot_points)) // (num_link_points)
    link_joint_map = jnp.array([0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 6])
    fks = panda_model.fk_fn(qs[q_idx])
    pos_jac = []
    for i, joint in enumerate(panda_model.joints.values()):
        if joint.joint_type != "revolute": continue
        joint_idx = i+1
        p_frame = fks[joint_idx][-3:]
        rot_axis = SE3(fks[joint_idx]).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, point - p_frame)
        pos_jac.append(lin_vel)
    point_jac = jnp.vstack(pos_jac).T
    masking = np.tile(np.arange(7),3).reshape(-1,7)
    masking = jnp.where(masking > link_joint_map[link_idx], 0, 1)
    return point_jac * masking

In [756]:
body_points[top4_indices]

Array([[ 0.48613888, -0.1033943 ,  0.28012142],
       [ 0.53716546, -0.09549025,  0.28453344],
       [ 0.5810802 , -0.09781151,  0.2921302 ],
       [ 0.48826456, -0.05896187,  0.28394106]], dtype=float32)

In [764]:
point_jacs = jax.vmap(point_jacobian, in_axes=(None, 0, 0))(jnp.array(qs), body_points[top4_indices], top4_indices)

In [784]:
repulsive_grad = jax.jacrev(env.distances)(body_points[top4_indices])

In [608]:
link = 3
idx_point = 10

In [620]:
surface_point = np.ones(3)
fks = panda_model.fk_fn(q)
pos_jac = []
for i, joint in enumerate(panda_model.joints.values()):
    if joint.joint_type != "revolute": continue
    joint_idx = i+1
    p_frame = fks[joint_idx][-3:]
    rot_axis = SE3(fks[joint_idx]).as_matrix()[:3, 2]
    lin_vel = jnp.cross(rot_axis, surface_point - p_frame)
    pos_jac.append(lin_vel)
point_jac = jnp.vstack(pos_jac).T.at[:,link:].set(0.)
point_jac

Array([[-1.        ,  0.66697174, -0.9989123 ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.00612833,  0.971879  ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -1.0091455 ,  0.04052969,  0.        ,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [610]:
panda_model.jac_fn(q).shape

(6, 7)

In [546]:
tic = time.perf_counter()
body_points = jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
body_pc.reload(points=body_points, size=0.01)
toc = time.perf_counter()
print(toc-tic)

0.009939231007592753


In [496]:
body_pc = PointCloud(world.vis, "body_pc", body_points, 0.02, "blue")

In [505]:
q_pick, q_place = qs[idx_pick], qs[idx_place]

In [599]:
obj_points = farthest_point_sampling(obj_start.mesh.sample(100), 15)
obj_pc = PointCloud(world.vis, "obj_pc", obj_points, 0.02, "green")
def get_object_points(q):
    ee, _ = get_ee_fk_jac(q)
    grasp_pose = grasp_reconst(grasp)
    obj_pose = ee @ grasp_pose.inverse()
    obj_points_q = jax.vmap(obj_pose.apply)(obj_points)
    return obj_points_q
#obj_pc.reload(points=obj_points_q)

Array([-6.316279 , -2.021728 , -1.417364 , -1.2467048, -3.4943879,
       -3.5216036, -6.7881074], dtype=float32)

In [390]:
q_pick = qsol[:7]
grasp = builder2.param_dict["grasp"]
p_start = builder2.param_dict["p_start"]

#kin_error_jac_fn(q_pick, grasp, p_start)
ee_pose, geom_jac = get_ee_fk_jac(q_pick)
obj_pose = SE3(to_wxyzxyz(p_start))
target_pose = obj_pose @ grasp_reconst(grasp)
err_rot = ee_pose.rotation().inverse() @ target_pose.rotation()
R_ee = ee_pose.rotation().as_matrix()
B = get_rotvec_angvel_map(err_rot.log())

In [392]:
get_rotvec_angvel_map(np.array([0.001,0,0.]))

Array([[ 1.000000e+00,  0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  1.048576e+00,  5.000000e-04],
       [ 0.000000e+00, -5.000000e-04,  1.048576e+00]], dtype=float32)

In [396]:
ipopt2.add_option("acceptable_iter", 2)
ipopt2.add_option("acceptable_tol", 0.1) #release
ipopt2.add_option("acceptable_obj_change_tol", 0.0001)
ipopt2.add_option("acceptable_dual_inf_tol", 1.) 
ipopt2.add_option('mu_strategy', 'adaptive')

qs_init = jnp.tile(panda.neutral, 2)
qsol, info = ipopt2.solve(qs_init)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       84
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       14
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       14
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.56e+00 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [399]:
panda.set_joint_angles(qsol[:7])

In [400]:
panda.set_joint_angles(qsol[7:])

In [247]:
qsol[7:]

array([ 0.07027957,  0.09895435, -0.09649118, -2.27088913, -1.55379442,
        1.61022823,  0.01779843])

In [246]:
panda.set_joint_angles(qsol[7:])

In [503]:
np.hstack([p_body, rot_body.log()])

array([-0.11530018,  0.2158832 , -0.1962212 ,  0.24268597, -0.00360402,
       -0.77968186], dtype=float32)

In [491]:

Rt = jnp.block([[R.T, np.zeros((3,3))],[np.zeros((3,3)), R.T]])

In [ ]:
E = get_rotvec_angvel_map(rot_body.log())

Array([6.1263680e-01, 4.6566129e-10, 5.5448973e-01], dtype=float32)

In [ ]:
q = panda.neutral
panda.set_joint_angles(q)
ee_pose, geom_jac = get_ee_fk_jac(q)
frame.set_pose(SE3(to_wxyzxyz(ee_posevec)))

target_pose = (obj_goal.pose @ grasp_reconst(grasp))
target_posevec = to_posevec(target_pose.parameters())
frame2.set_pose(SE3(to_wxyzxyz(target_posevec)))

ee_posevec, jac = get_ee_fk_jac(q)
err = target_posevec - ee_posevec
d = - jnp.linalg.pinv(jac.T@jac).T@(-jac.T)@err

q += d*0.05
panda.set_joint_angles(q)
frame.set_pose(SE3(to_wxyzxyz(ee_posevec)))

In [395]:
panda.set_joint_angles(panda.neutral)

In [232]:
zflip = SE3.from_rotation(SO3.from_z_radians(jnp.pi))

def kin_error_fn(q, grasp, p_obj):
    ee_pose, jac = get_ee_fk_jac(q)
    obj_pose = SE3(to_wxyzxyz(p_obj))
    grasp_pose = obj_pose @ grasp_reconst(grasp) @ zflip
    grasp_posevec = to_posevec(grasp_pose.parameters())
    err = grasp_posevec - posevec_ee
    return err

def kin_error_jac_fn(q, grasp, p_obj):
    _, jac = get_ee_fk_jac(q)
    return - jac

def debug_obj2(q, grasp, p_obj):
    panda.set_joint_angles(q)
    posevec_ee, jac = get_ee_fk_jac(q)
    #obj_pose = SE3(to_wxyzxyz(p_obj))
    #frame2.set_pose(SE3(to_wxyzxyz(posevec_ee)))
    #grasp_pose = obj_pose @ grasp_reconst(grasp) @ zflip
    #frame.set_pose(grasp_pose)
    #pc.reload(points=np.vstack(pcs))
    time.sleep(0.1)
    return 0.

builder2 = SparseIPOPTBuilder()

builder2.add_variable("q_pick", 7, panda.lb, panda.ub)
#builder2.add_variable("q_place", 7, panda.lb, panda.ub)
builder2.add_parameter("p_start", to_posevec(obj_start.pose.parameters()))
#builder2.add_parameter("p_goal", to_posevec(obj_goal.pose.parameters()))
builder2.add_parameter("grasp", xsol)

builder2.register_fn("kin_constr", [7, 3, 6], 6,
                          kin_error_fn, [kin_error_jac_fn, None, None])
builder2.register_fn("obj_debug", [7, 3, 6], 1,
                          debug_obj2, [None, None, None])
builder2.add_objective(["q_pick", "grasp", "p_start"], "obj_debug")
builder2.add_constr("kin", ["q_pick", "grasp", "p_start"], "kin_constr",
                   0., 0.)

In [233]:
builder2.freeze()
ipopt2 = builder2.build(compile_obj=False)

TypeError: unsupported operand type(s) for -: 'BatchTracer' and 'SE3'

In [231]:
qsol, info = ipopt2.solve(panda.neutral)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       42
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.56e+00 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

NameError: name 'frame2' is not defined

In [16]:
import PyCeres

In [449]:
class IK(PyCeres.CostFunction):
    def __init__(self, grasp, obj_pose, viz=False):
        super().__init__()
        self.set_num_residuals(6)
        self.set_parameter_block_sizes([7])
        self.target_pose = (obj_pose @ grasp_reconst(grasp))
        self.viz = viz
    
    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0]
        ee_pose, geom_jac = get_ee_fk_jac(q)
        R_ee = ee_pose.rotation().as_matrix()
        err_t = R_ee.T@(self.target_pose.translation() - ee_pose.translation()) #body
        err_r = ee_pose.rotation().inverse() @ self.target_pose.rotation()
        err = jnp.hstack([err_t, err_r.log()])

        if (jacobians != None):
            B = get_rotvec_angvel_map(err_r.log())
            jac_t = - R_ee.T @ geom_jac[:3]
            jac_r = - B @ R_ee.T @ geom_jac[3:]
            jac = jnp.vstack([jac_t, jac_r]) 
            jacobians[0][:] = jac.flatten()
        residuals[:] = err
        if self.viz:
            panda.set_joint_angles(q)
            time.sleep(0.1)
        return True
    
class DistanceBetween(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(7)
        self.set_parameter_block_sizes([7,7])
        self.weight = 0.1

    def Evaluate(self, parameters, residuals, jacobians):
        q1, q2 = parameters[0], parameters[1]
        err = (q1-q2)
        residuals[:] = err*self.weight
        if (jacobians != None):
            jacobians[0][:] = np.eye(7).flatten() * self.weight
            jacobians[1][:] = - np.eye(7).flatten() * self.weight
        return True

class DistanceFrom(PyCeres.CostFunction):
    def __init__(self, target_config):
        super().__init__()
        self.target_config = target_config
        self.set_num_residuals(7)
        self.set_parameter_block_sizes([7])
        self.weight = 0.1

    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0]
        err = (self.target_config - q)
        residuals[:] = err*self.weight
        if (jacobians != None):
            jacobians[0][:] = - np.eye(7).flatten() * self.weight
        return True

# def point_jacobian(q, point, link_idx):
#     link_joint_map = jnp.array([0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 6])
#     fks = panda_model.fk_fn(q)
#     pos_jac = []
#     for i, joint in enumerate(panda_model.joints.values()):
#         if joint.joint_type != "revolute": continue
#         joint_idx = i+1
#         p_frame = fks[joint_idx][-3:]
#         rot_axis = SE3(fks[joint_idx]).as_matrix()[:3, 2]
#         lin_vel = jnp.cross(rot_axis, point - p_frame)
#         pos_jac.append(lin_vel)
#     point_jac = jnp.vstack(pos_jac).T
#     masking = np.tile(np.arange(7),3).reshape(-1,7)
#     masking = jnp.where(masking > link_joint_map[link_idx], 0, 1)
#     return point_jac * masking

# pc = PointCloud(world.vis, "pc", np.zeros((10,3)), 0.01, "blue")
# num_link_points = 20
# class Collision(PyCeres.CostFunction):
#     def __init__(self, viz=False):
#         super().__init__()
#         self.set_num_residuals(1)
#         self.set_parameter_block_sizes([7])
#         self.safe_dist = 0.05
#         self.weight = 1.
#         self.viz = viz
    
#     def Evaluate(self, parameters, residuals, jacobians):
#         q = parameters[0]

#         robot_pc = panda.get_surface_points_fn(q)
#         distances = env.distances(robot_pc)
#         idx = distances.argmin()
#         min_dist = distances[idx]

#         if self.viz:
#             panda.set_joint_angles(q)
#             pc.reload(points=robot_pc, color="blue")
#             time.sleep(0.1)
        
#         if min_dist > self.safe_dist:
#             residuals[:] = 0.
#             if (jacobians != None):
#                 jacobians[0][:] = np.zeros(7)
#             return True
#         else:
#             residuals[:] = min_dist - self.safe_dist
#             if (jacobians != None):
#                 point = robot_pc[idx]
#                 link_idx = idx // num_link_points
#                 jac_point = point_jacobian(q, point, link_idx)
#                 repulsive_grad = jax.grad(env.distance)(point)
#                 jacobians[0][:] = repulsive_grad @ jac_point
#             return True

class CollisionObject(PyCeres.CostFunction):
    def __init__(self, grasp, obj_points, viz=False):
        super().__init__()
        self.grasp_pose = grasp_reconst(grasp)
        self.obj_points = obj_points
        self.set_num_residuals(1)
        self.set_parameter_block_sizes([7])
        self.safe_dist = 0.05
        self.weight = 1.
        self.viz = viz
    
    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0]
        
        fks = panda_model.fk_fn(q)
        obj_pose = SE3(fks[-1]) @ self.grasp_pose.inverse()
        obj_points = jax.vmap(obj_pose.apply)(self.obj_points)
        distances = env.distances(obj_points)
        idx = distances.argmin()
        min_dist = distances[idx]

        if self.viz:
            pc.reload(points=obj_points, color="green")
            time.sleep(0.1)
            
        if min_dist > self.safe_dist:
            residuals[:] = 0.
            if (jacobians != None):
                jacobians[0][:] = np.zeros(7)
            return True
        else:
            residuals[:] = min_dist - self.safe_dist
            if (jacobians != None):
                point = obj_points[idx]
                jac_point = point_jacobian(q, point, -1)
                repulsive_grad = jax.grad(env.distance)(point)
                jacobians[0][:] = repulsive_grad @ jac_point
            return True
        
        

In [468]:
points = jax.vmap(panda.get_surface_points_fn)(qs)

In [481]:
q1 = qs[0]
safe_dist = 0.05

In [817]:
def point_jacobian(point, last_joint_idx, fks):
    pos_jac = []
    for i, joint in enumerate(panda_model.joints.values()):
        if joint.joint_type != "revolute": continue
        joint_idx = i+1
        p_frame = fks[joint_idx][-3:]
        rot_axis = SE3(fks[joint_idx]).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, point - p_frame)
        pos_jac.append(lin_vel)
    point_jac = jnp.vstack(pos_jac).T
    masking = np.tile(np.arange(7),3).reshape(-1,7)
    masking = jnp.where(masking > last_joint_idx, 0, 1)
    return point_jac * masking

#get links_points
links_points = []
for link in panda_model.links.values():
    if not link.has_mesh: continue
    if not panda_model.is_floating and link == panda_model.root_link: continue
    links_points.append(link.surface_points)
links_points = np.array(links_points)
fk_assign = lambda wxyzxyz, link_points: jax.vmap(SE3(wxyzxyz).apply)(link_points)

@jax.jit
def get_col_vj(q):
    def no_col(point, link_idx, fks):
        return np.zeros(7)
    def col(point, link_idx, fks):
        last_joint_idx = jnp.maximum(link_idx, 6)
        jac_point = point_jacobian(point, last_joint_idx, fks)
        repulsive_grad = jax.grad(env.distance)(point)
        return repulsive_grad @ jac_point
    
    fks = panda_model.fk_fn(q)
    assigned_points = jax.vmap(fk_assign)(fks[1:-1], links_points)
    distances = jax.vmap(env.distances)(assigned_points)
    link_idx, point_idx = jnp.unravel_index(
        distances.argmin(), distances.shape)
    point = assigned_points[link_idx, point_idx]
    min_dist = distances[link_idx, point_idx] - safe_dist
    is_penet = min_dist < 0.
    penet = jnp.where(is_penet, min_dist, 0.)
    jac = jax.lax.cond(is_penet, col, no_col, point, link_idx, fks)
    return penet, jac

obj_points = farthest_point_sampling(obj_start.mesh.sample(200), 20)
@jax.jit
def get_col_obj_vj(q, grasp_pose:SE3):
    def no_col(point, fks):
        return np.zeros(7)
    def col(point, fks):
        last_joint_idx = 6 #calculate full jacobian
        jac_point = point_jacobian(point, last_joint_idx, fks)
        repulsive_grad = jax.grad(env.distance)(point)
        return repulsive_grad @ jac_point
    
    fks = panda_model.fk_fn(q)
    obj_pose = SE3(fks[-1]) @ grasp_pose.inverse()
    assigned_points = jax.vmap(obj_pose.apply)(obj_points)
    distances = env.distances(assigned_points)
    idx = distances.argmin()
    point = assigned_points[idx]
    min_dist = distances[idx] - safe_dist
    is_penet = min_dist < 0.
    penet = jnp.where(is_penet, min_dist, 0.)
    jac = jax.lax.cond(is_penet, col, no_col, point, fks)
    return penet, jac

In [821]:
penets, jacs = jax.vmap(get_col_obj_vj, in_axes=(0, None))(qs, grasp_reconst(grasp))

In [825]:
class CollisionBatch(PyCeres.CostFunction):
    def __init__(self, num_traj, viz=False):
        super().__init__()
        self.set_num_residuals(num_traj)
        self.set_parameter_block_sizes([7]*num_traj)
        self.safe_dist = 0.05
        self.weight = 1.
        self.viz = viz
    
    def Evaluate(self, parameters, residuals, jacobians):
        qs = jnp.array(parameters)
        penets, jacs = jax.vmap(get_col_vj)(qs)
        residuals[:] = penets
        if (jacobians != None):
            jacobians[:] = jacs
        return True
    
class CollisionObjectBatch(PyCeres.CostFunction):
    def __init__(self, grasp, num_hold, viz=False):
        super().__init__()
        self.set_num_residuals(num_hold)
        self.set_parameter_block_sizes([7]*num_hold)
        self.grasp_pose = grasp_reconst(grasp)
        self.safe_dist = 0.05
        self.weight = 1.
        self.viz = viz
    
    def Evaluate(self, parameters, residuals, jacobians):
        qs = jnp.array(parameters)
        penets, jacs = jax.vmap(get_col_obj_vj)(qs)
        residuals[:] = penets
        if (jacobians != None):
            jacobians[:] = jacs
        return True

In [845]:
num_mid_point = 3
num_traj = num_mid_point*3 + 2
qs = np.array([panda.neutral.copy() for i in range(num_traj)])
#qs = [*qs]
idx_pick = num_mid_point
idx_place = idx_pick + num_mid_point + 1

grasp = xsol

In [846]:
problem = PyCeres.Problem()
feat_ik_pick = IK(grasp, obj_start.pose, viz=False)
feat_ik_place = IK(grasp, obj_goal.pose)
feat_dist_between = DistanceBetween()
feat_dist_from_home = DistanceFrom(panda.neutral)
feat_col = CollisionBatch(num_traj)
feat_col_obj = CollisionObjectBatch(grasp, 3)

for i in range(num_traj-1):
    problem.AddResidualBlock(feat_dist_between, None, qs[i], qs[i+1])
# for i in range(num_traj):
#     problem.AddResidualBlock(feat_col, None, qs[i])
# for i in np.arange(idx_pick+1, idx_place):
#     problem.AddResidualBlock(feat_col_obj, None, qs[i])
problem.AddResidualBlock(feat_col, None, qs.reshape(1, -1))
problem.AddResidualBlock(feat_col_obj, None, *qs[idx_pick+1:idx_place])
problem.AddResidualBlock(feat_dist_from_home, None, qs[0])
problem.AddResidualBlock(feat_dist_from_home, None, qs[-1])
problem.AddResidualBlock(feat_ik_pick, None, qs[idx_pick])
problem.AddResidualBlock(feat_ik_place, None, qs[idx_place])

options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.parameter_tolerance = 1e-3
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()

: 

: 

In [ ]:
PyCeres.Solve(options, problem, summary)

In [789]:
qs = np.random.uniform(panda.lb, panda.ub, size=(11,7))
qs = jnp.array(qs)
parameters = qs
residuals = np.zeros(11)
jacobians = [*np.zeros_like(qs)]
feat_col.Evaluate(parameters, residuals, jacobians)

In [796]:
tic = time.perf_counter()
get_col_vj(q)
toc = time.perf_counter()
toc-tic

0.001087398995878175

In [740]:
q = panda.get_random_config()
panda.set_joint_angles(q)
tic = time.perf_counter()
penet, jac = get_col_vj(q)
toc = time.perf_counter()
toc-tic

0.0005161419976502657

In [177]:
# collision test
# q = panda.get_random_config()
# panda.set_joint_angles(q)

# problem = PyCeres.Problem()
# problem.AddResidualBlock(feat_col, None, q)

# options = PyCeres.SolverOptions()
# options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
# options.minimizer_progress_to_stdout = True
# summary = PyCeres.Summary()
# PyCeres.Solve(options, problem, summary)

In [165]:
# parameters = [q]
# residuals = np.zeros(1)
# jacobians = [np.zeros(7)]
# feat_col.Evaluate(parameters, residuals, jacobians)

True

In [361]:
obj_points = farthest_point_sampling(obj_start.mesh.sample(200), 20)

In [451]:
problem = PyCeres.Problem()
feat_ik_pick = IK(grasp, obj_start.pose, viz=False)
feat_ik_place = IK(grasp, obj_goal.pose)
feat_dist_between = DistanceBetween()
feat_dist_from_home = DistanceFrom(panda.neutral)
feat_col = Collision(viz=True)
feat_col_obj = CollisionObject(grasp, obj_points, viz=True)

for i in range(num_traj-1):
    problem.AddResidualBlock(feat_dist_between, None, qs[i], qs[i+1])
for i in range(num_traj):
    problem.AddResidualBlock(feat_col, None, qs[i])
for i in np.arange(idx_pick+1, idx_place):
    problem.AddResidualBlock(feat_col_obj, None, qs[i])

problem.AddResidualBlock(feat_dist_from_home, None, qs[0])
problem.AddResidualBlock(feat_dist_from_home, None, qs[-1])
problem.AddResidualBlock(feat_ik_pick, None, qs[idx_pick])
problem.AddResidualBlock(feat_ik_place, None, qs[idx_place])

options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.parameter_tolerance = 1e-3
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.074764e+00    0.00e+00    1.49e+00   0.00e+00   0.00e+00  1.00e+04        0    2.29e+00    2.29e+00
   1  1.877118e+00    1.98e-01    1.78e+00   5.88e+00   9.63e-02  6.55e+03        1    4.66e+00    6.95e+00
   2  4.975296e-01    1.38e+00    7.35e-01   3.69e+00   7.41e-01  7.37e+03        1    4.44e+00    1.14e+01
   3  3.477292e-01    1.50e-01    6.61e-01   3.15e+00   3.17e-01  7.03e+03        1    4.51e+00    1.59e+01
   4  1.320875e-01    2.16e-01    3.35e-01   2.68e+00   6.57e-01  7.25e+03        1    4.48e+00    2.04e+01
   5  4.349268e-02    8.86e-02    1.32e-01   1.46e+00   7.41e-01  8.16e+03        1    4.56e+00    2.49e+01
   6  1.316158e-02    3.03e-02    1.80e-02   6.80e-01   9.99e-01  2.45e+04        1    4.41e+00    2.93e+01
   7  2.460875e-02   -1.14e-02    1.80e-02   4.27e-01  -1.83e+01  1.22e+04        1    2.05e+00    3.14e+01
   8  2.460898e-02   -1.14e-

In [452]:
qs_vis = [panda.neutral.copy(), *qs, panda.neutral.copy()]
i = 0

In [465]:
panda.set_joint_angles(qs_vis[i]) #qs[4]
i+= 1

In [165]:
class IK(PyCeres.CostFunction):
    def __init__(self, grasp):
        super().__init__()
        self.set_num_residuals(6)
        self.set_parameter_block_sizes([7])
        target_pose = (obj_goal.pose @ grasp_reconst(grasp))
        target_posevec = to_posevec(target_pose.parameters())
        self.ik_target = target_posevec
    
    def Evaluate(self, parameters, residuals, jacobians):
        #parameters = [q]
        #residuals = [err]
        #jacobians = [jac_q]
        q = parameters[0]
        ee_posevec, jac = get_ee_fk_jac(q)
        err = self.ik_target - ee_posevec
        if (jacobians != None):
            jacobians[0][:] = -jac.flatten()
        residuals[:] = err
        panda.set_joint_angles(q)
        time.sleep(0.1)
        return True    
    
class IK2(PyCeres.CostFunction):
    def __init__(self, grasp, obj_poses):
        super().__init__()
        self.set_num_residuals(12)
        self.set_parameter_block_sizes([7, 7])
        target_posevecs = []
        for obj_pose in obj_poses:
            target_pose = (obj_pose @ grasp_reconst(grasp))
            target_posevec = to_posevec(target_pose.parameters())
            target_posevecs.append(target_posevec)
        self.target_posevecs = jnp.hstack(target_posevecs)
        frame2.set_pose(SE3(to_wxyzxyz(target_posevecs[1])))
    
    def Evaluate(self, parameters, residuals, jacobians):
        #parameters = [q1, q2]
        #residuals = [err1_err2]
        #jacobians = [jac_q1, jac_q2]
        q1, q2 = parameters[0], parameters[1]
        ee_posevecs, jacs = jax.vmap(get_ee_fk_jac)(jnp.vstack([q1,q2]))
        err = self.target_posevecs - jnp.hstack(ee_posevecs)
        if (jacobians != None):
            jac1 = jnp.pad(-jacs[0], ((0,6),(0,0))).flatten()
            jac2 = jnp.pad(-jacs[1], ((6,0),(0,0))).flatten()
            jacobians[0][:] = jac1
            jacobians[1][:] = jac2
        residuals[:] = err
        # panda.set_joint_angles(q1)
        # time.sleep(0.1)
        panda.set_joint_angles(q2)
        frame.set_pose(SE3(to_wxyzxyz(ee_posevecs[1])))
        time.sleep(1.)
        return True    

# mid_steps = 2
# pick_idx = mid_steps
# place_idx = mid_steps*2+1
# class TravelledDistance(PyCeres.CostFunction):
#     def __init__(self):
#         super().__init__()
#         self.q0 = panda.neutral
#         q_num = (mid_steps*3+2)
#         in_dim = 7*q_num
#         self.set_num_residuals(in_dim+1)
#         self.set_parameter_block_sizes([7]*q_num)
#         self.Iblock = np.vstack([-np.eye(7), np.eye(7)])
    
#     def Evaluate(self, parameters, residuals, jacobians):
#         q1, q2 = parameters[0], parameters[1]
#         qerr1 = self.q0 - q1
#         qerr2 = q1 - q2
#         qerr3 = q2 - self.q0
#         qerr = np.hstack([qerr1, qerr2, qerr3])
#         if (jacobians != None):
#             jacobians[0][:] = jnp.pad(self.Iblock, ((0,7),(0,0))).flatten()*0.001
#             jacobians[1][:] = jnp.pad(self.Iblock, ((7,0),(0,0))).flatten()*0.001
#         residuals[:] = qerr*0.001
#         return True

class ConfigDiff(PyCeres.CostFunction):
    def __init__(self, q):
        super().__init__()
        self.q_target = q
        self.set_num_residuals(7)
        self.set_parameter_block_sizes([7])
        self.weight = 0.01

    def Evaluate(self, parameters, residuals, jacobians):
        q = parameters[0]
        qerr = (self.q_target - q) * self.weight
        residuals[:] = qerr
        if (jacobians != None):
            jacobians[0][:] = - np.eye(7).flatten() * self.weight
        return True

class ConfigDiffBetween(PyCeres.CostFunction):
    def __init__(self):
        super().__init__()
        self.set_num_residuals(7)
        self.set_parameter_block_sizes([7,7])
        self.weight = 0.01

    def Evaluate(self, parameters, residuals, jacobians):
        q1, q2 = parameters[0], parameters[1]
        qerr = (q1-q2)*self.weight
        residuals[:] = qerr
        if (jacobians != None):
            jacobians[0][:] = np.eye(7).flatten() * self.weight
            jacobians[1][:] = - np.eye(7).flatten() * self.weight
        return True


In [163]:
mid_steps = 2
pick_idx = mid_steps
place_idx = mid_steps*2+1
q_num = mid_steps*3+2
qs = np.tile(panda.neutral, q_num).reshape(-1, 7)

In [164]:
problem = PyCeres.Problem()
feat_dist = ConfigDiffBetween()
feat_boundary = ConfigDiff(panda.neutral)
feat_ik2 = IK2(grasp, [obj_start.pose, obj_goal.pose])
for i in range(0, q_num-1):
    problem.AddResidualBlock(feat_dist, None, qs[i], qs[i+1])
problem.AddResidualBlock(feat_boundary, None, qs[0])
problem.AddResidualBlock(feat_boundary, None, qs[-1])
problem.AddResidualBlock(feat_ik2, None, qs[pick_idx], qs[place_idx])

options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.062161e+01    0.00e+00    4.80e+00   0.00e+00   0.00e+00  1.00e+04        0    1.01e+00    1.01e+00
   1  4.185447e+00    1.64e+01    1.92e+00   1.54e+01   7.97e-01  1.27e+04        1    2.03e+00    3.05e+00
   2  3.422278e+00    7.63e-01    2.63e+00   9.69e+00   1.82e-01  1.01e+04        1    2.05e+00    5.10e+00
   3  4.522949e+00   -1.10e+00    2.63e+00   5.33e+01  -3.32e-01  5.04e+03        1    1.04e+00    6.13e+00
   4  6.379828e+00   -2.96e+00    2.63e+00   4.51e+01  -9.02e-01  1.26e+03        1    1.01e+00    7.15e+00
   5  5.553830e+00   -2.13e+00    2.63e+00   2.35e+01  -6.94e-01  1.57e+02        1    1.02e+00    8.16e+00
   6  9.432352e-01    2.48e+00    1.21e+00   4.70e+00   8.96e-01  3.14e+02        1    2.10e+00    1.03e+01
   7  1.530258e+01   -1.44e+01    1.21e+00   4.86e+00  -1.54e+01  1.57e+02        1    1.03e+00    1.13e+01
   8  1.591114e+01   -1.50e+

TypeError: Cannot determine dtype of [[-2.6727405 -1.2061212  4.1728516 -6.379529  -4.1083183  5.074463
   9.221579 ]]

In [143]:
i = 0

In [152]:
panda.set_joint_angles(qs[i])
i += 1

IndexError: index 8 is out of bounds for axis 0 with size 8

In [407]:
q = panda.neutral
panda.set_joint_angles(q)
ee_posevec, jac = get_ee_fk_jac(q)
frame.set_pose(SE3(to_wxyzxyz(ee_posevec)))

target_pose = (obj_goal.pose @ grasp_reconst(grasp))
target_posevec = to_posevec(target_pose.parameters())
frame2.set_pose(SE3(to_wxyzxyz(target_posevec)))

ee_posevec, jac = get_ee_fk_jac(q)
err = target_posevec - ee_posevec
d = - jnp.linalg.pinv(jac.T@jac).T@(-jac.T)@err

q += d*0.05
panda.set_joint_angles(q)
frame.set_pose(SE3(to_wxyzxyz(ee_posevec)))

In [408]:
target_pose = (obj_goal.pose @ grasp_reconst(grasp))
target_posevec = to_posevec(target_pose.parameters())
frame2.set_pose(SE3(to_wxyzxyz(target_posevec)))

In [466]:
ee_posevec, jac = get_ee_fk_jac(q)
err = target_posevec - ee_posevec
d = - jnp.linalg.pinv(jac.T@jac).T@(-jac.T)@err

q += d*0.05
panda.set_joint_angles(q)
frame.set_pose(SE3(to_wxyzxyz(ee_posevec)))

In [182]:
d

Array([ 1.1487997, -2.256655 ,  1.1487826, -4.188451 , -6.3163266,
        4.429671 ,  4.023236 ], dtype=float32)

In [166]:
problem = PyCeres.Problem()
q = panda.neutral.copy()
# feat_dist = ConfigDiffBetween()
# feat_boundary = ConfigDiff(panda.neutral)
feat_ik = IK(grasp)
# for i in range(0, q_num-1):
#     problem.AddResidualBlock(feat_dist, None, qs[i], qs[i+1])
# problem.AddResidualBlock(feat_boundary, None, qs[0])
# problem.AddResidualBlock(feat_boundary, None, qs[-1])
problem.AddResidualBlock(feat_ik, None, q)

options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.804322e+01    0.00e+00    4.80e+00   0.00e+00   0.00e+00  1.00e+04        0    1.45e-01    1.45e-01
   1  3.617087e+00    1.44e+01    1.93e+00   1.00e+01   8.00e-01  1.27e+04        1    2.11e-01    3.56e-01
   2  7.131869e+00   -3.51e+00    1.93e+00   6.44e+00  -9.72e-01  6.37e+03        1    1.08e-01    4.65e-01
   3  7.135639e+00   -3.52e+00    1.93e+00   6.42e+00  -9.73e-01  1.59e+03        1    1.21e-01    5.85e-01
   4  7.170883e+00   -3.55e+00    1.93e+00   6.30e+00  -9.83e-01  1.99e+02        1    1.16e-01    7.01e-01
   5  8.156652e+00   -4.54e+00    1.93e+00   5.46e+00  -1.26e+00  1.24e+01        1    1.05e-01    8.06e-01
   6  1.946371e+01   -1.58e+01    1.93e+00   2.63e+00  -4.78e+00  3.89e-01        1    1.20e-01    9.26e-01
   7  1.925257e+00    1.69e+00    1.49e+00   6.80e-01   9.59e-01  1.17e+00        1    2.41e-01    1.17e+00
   8  7.193471e-01    1.21e+

In [89]:
qs = np.tile(panda.neutral).reshape(-1, 7)
q2 = np.array(panda.neutral)
feat_ik2 = IK2(grasp, [obj_start.pose, obj_goal.pose])
feat_dist = TravelledDistance()
panda.set_joint_angles(q1)

In [90]:
problem = PyCeres.Problem()
problem.AddResidualBlock(feat_ik2, None, q1, q2)
problem.AddResidualBlock(feat_dist, None, q1, q2)
options = PyCeres.SolverOptions()
options.linear_solver_type = PyCeres.LinearSolverType.SPARSE_NORMAL_CHOLESKY
options.minimizer_progress_to_stdout = True
summary = PyCeres.Summary()
PyCeres.Solve(options, problem, summary)

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.062161e+01    0.00e+00    4.80e+00   0.00e+00   0.00e+00  1.00e+04        0    2.19e-01    2.19e-01
   1  4.194706e+00    1.64e+01    1.93e+00   1.02e+01   7.97e-01  1.26e+04        1    4.68e-01    6.87e-01
   2  6.949550e+00   -2.75e+00    1.93e+00   6.52e+00  -6.57e-01  6.32e+03        1    2.26e-01    9.13e-01
   3  7.074964e+00   -2.88e+00    1.93e+00   6.50e+00  -6.87e-01  1.58e+03        1    2.47e-01    1.16e+00
   4  7.203941e+00   -3.01e+00    1.93e+00   6.38e+00  -7.17e-01  1.97e+02        1    2.21e-01    1.38e+00
   5  8.223922e+00   -4.03e+00    1.93e+00   5.53e+00  -9.64e-01  1.23e+01        1    2.39e-01    1.62e+00
   6  1.950366e+01   -1.53e+01    1.93e+00   2.70e+00  -3.94e+00  3.86e-01        1    2.30e-01    1.85e+00
   7  2.096153e+00    2.10e+00    1.49e+00   7.29e-01   9.69e-01  1.16e+00        1    4.61e-01    2.31e+00
   8  7.523395e-01    1.34e+

In [87]:
panda.set_joint_angles(q2)

In [503]:
init[0]

array([-0.17116671, -0.0994386 , -0.31581191, -2.00006747,  0.07544014,
        2.03134143, -1.35211934])

In [497]:
panda.set_joint_angles()

In [464]:
print(summary.BriefReport())

TypeError: descriptor '__dir__' of 'object' object needs an argument

In [396]:
q = panda.neutral.copy()


panda.set_joint_angles(panda.neutral)

In [454]:
ee, jac = get_ee_fk_jac(q)
err = target_posevec - ee
grad = - jac.T @ err
hess = jac.T@jac
d = - jnp.linalg.pinv(hess) @ grad

q = q + d*0.1

panda.set_joint_angles(q)

In [19]:
qsol, info = ipopt2.solve(panda.neutral)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       42
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.39e+00 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

KeyboardInterrupt: 

SE3(wxyz=[ 0.00842    -0.68869996 -0.72375    -0.04238   ], xyz=[-0.00169     0.05138     0.10241999])

In [19]:
x = jnp.zeros(3)
zip_x_ndo = zip(builder.xnames, builder.xdims, builder.xoffsets)
input_dict = {name:x[offset:offset+dim] 
                    for name, dim, offset in zip_x_ndo}
input_dict.update(builder.param_dict)
input_names = [x.name for x in builder.obj_info.inputs]
inputs = [input_dict[name] for name in input_names]

In [16]:
list(zip_x_ndo)

[('g_pick', 3, 0)]

In [17]:
[x.name for x in builder.obj_info.inputs]

['g_pick', 'p_start', 'p_goal']

In [43]:
builder = SparseIPOPTBuilder()

builder.add_variable("g_pick", 3, -1., 1.)
builder.add_variable("g_place", 3, -1., 1.)
builder.add_variable("p_ho", 6, ws_lb, ws_ub)
builder.add_parameter("p_start", to_posevec(obj_start.pose.parameters()))
builder.add_parameter("p_goal", to_posevec(obj_goal.pose.parameters()))

builder.register_fn("grasp_constr", [3], 1,
                          grasp_constr_fn, grasp_constr_jac_fns)
builder.register_fn("manip_constr", [3, 6], 1,
                          manip_constr_fn, manip_constr_jac_fns)
builder.register_fn("dist_constr", [3, 3, 6, 6, 6], 4,
                          dist_constr_fn, dist_constr_jac_fns)

builder.add_constr("grasp_prob_pick", ["g_pick"], "grasp_constr",
                   1., np.inf)
builder.add_constr("grasp_prob_place", ["g_place"], "grasp_constr",
                   1., np.inf)
builder.add_constr("manip_pick", ["g_pick", "p_start"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("manip_place", ["g_place", "p_goal"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("manip_ho_1", ["g_pick", "p_ho"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("manip_ho_2", ["g_place", "p_ho"], "manip_constr",
                   0.3, np.inf)
builder.add_constr("dist", ["g_pick", "g_place", "p_ho", "p_start", "p_goal"], "dist_constr",
                   0.05, np.inf)
builder.freeze()

In [45]:
ipopt = builder.build(True)


Sparsity pattern:
00 01 02 -- -- -- -- -- -- -- -- -- 
-- -- -- 03 04 05 -- -- -- -- -- -- 
06 07 08 -- -- -- -- -- -- -- -- -- 
-- -- -- 09 10 11 -- -- -- -- -- -- 
12 13 14 -- -- -- 18 19 20 21 22 23 
-- -- -- 15 16 17 24 25 26 27 28 29 
30 31 32 42 43 44 54 55 56 57 58 59 
33 34 35 45 46 47 60 61 62 63 64 65 
36 37 38 48 49 50 66 67 68 69 70 71 
39 40 41 51 52 53 72 73 74 75 76 77 


In [ ]:
x = [np.zeros(3), np.zer]
ipopt.solve()

In [17]:
grasp = jnp.zeros(3)
posevec = jnp.zeros(6)



Array([0.4810451, 0.5339005], dtype=float32)

In [23]:
dist_constr_fn(grasp, grasp, posevec, posevec, posevec)

Array([0.26522437, 0.26522437, 0.26522437, 0.26522437], dtype=float32)